In [ ]:
!pip install sastrawi


In [ ]:
#import some libraries
import pandas as pd
import re
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory



#Import data
Data taked from google drive

In [ ]:
#connect google drive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/NLP

/content/drive/My Drive/NLP


In [ ]:
#import data
df = pd.read_csv("names.csv")
df.head(10)

,nama,arti
0,Angkasa,yang bermakna langit
1,Awan,"yang bermakna pendukung, langit"
2,Amat,yang bermakna mengamati
3,Ahyad,yang bermakna mulia
4,Aidil,yang bermakna jujur
5,Arief,yang bermakna berpengetahuan luas
6,Aman,yang bermakna damai
7,Aufa,yang bermakna kebenaran
8,Amien,yang bermakna harapan
9,Alam,yang bermakna dunia


In [ ]:
df.isnull().sum()

nama    0
arti    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nama    478 non-null    object
 1   arti    478 non-null    object
dtypes: object(2)
memory usage: 7.6+ KB


#preprosessing data



Cleaning data from dupliacates

In [ ]:
#clean dupilcate

df[df.duplicated(subset='nama')]

,nama,arti
189,Ihsan,"Kebaikan, kemurahan hati."
190,Ilyas,Nabi Ilyas yang terkenal dalam Al-Qur’an.
238,Batara,wibawa dewa
252,Bagas,kuat dan teguh
253,Bagaskoro,matahari bersinar
254,Bagya,"senang, bahagia"
255,Bajradaka,air intan
256,Bamantara,penguasa udara
257,Barata,menjauhi duniawi
258,Basiran,dermawan


In [ ]:
df_nodc = df.drop_duplicates(subset='nama')

Clean data from stopwords and punction

In [ ]:
sastrawi = StopWordRemoverFactory()
stopwordin = sastrawi.get_stop_words()

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('[/(){}\[\]\|@,;]',' ', text)
    text = re.sub('[^0-9a-z #+_]',' ', text)
    text = ' '.join(token for token in text.split() if token not in stopwordin)
    return text

In [ ]:
df_nodc['arti_clean'] = df_nodc['arti'].apply(clean_text)
df_nodc.head()

<ipython-input-50-8cf9f2309d8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodc['arti_clean'] = df_nodc['arti'].apply(clean_text)


,nama,arti,arti_clean
0,Angkasa,yang bermakna langit,bermakna langit
1,Awan,"yang bermakna pendukung, langit",bermakna pendukung langit
2,Amat,yang bermakna mengamati,bermakna mengamati
3,Ahyad,yang bermakna mulia,bermakna mulia
4,Aidil,yang bermakna jujur,bermakna jujur


In [ ]:
df_nodc.to_csv("df_nodc.csv")

# Bulid and trying Model

change data to vector wuth TFID

In [ ]:
user_response = pd.DataFrame({ 'nama':['-'],
                              'arti':['-'],
                               'arti_clean':['baik jujur dan tidak sombong']

})

df_bar = pd.concat([user_response, df_nodc]).reset_index()


In [ ]:
df_bar

,index,nama,arti,arti_clean
0,0,-,-,baik jujur dan tidak sombong
1,0,Angkasa,yang bermakna langit,bermakna langit
2,1,Awan,"yang bermakna pendukung, langit",bermakna pendukung langit
3,2,Amat,yang bermakna mengamati,bermakna mengamati
4,3,Ahyad,yang bermakna mulia,bermakna mulia
...,...,...,...,...
425,473,Yudanta,dalam perang,perang
426,474,Yudayana,panglima Perang,panglima perang
427,475,Yuwa,muda,muda
428,476,Yoda,bersifat perwira,bersifat perwira


In [ ]:
df_bar.pop('index')
df_bar

,nama,arti,arti_clean
0,-,-,baik jujur dan tidak sombong
1,Angkasa,yang bermakna langit,bermakna langit
2,Awan,"yang bermakna pendukung, langit",bermakna pendukung langit
3,Amat,yang bermakna mengamati,bermakna mengamati
4,Ahyad,yang bermakna mulia,bermakna mulia
...,...,...,...
425,Yudanta,dalam perang,perang
426,Yudayana,panglima Perang,panglima perang
427,Yuwa,muda,muda
428,Yoda,bersifat perwira,bersifat perwira


In [ ]:
#vectorisasi
TfidfVec= TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,)
tfidf= TfidfVec.fit_transform(df_bar['arti_clean'])
con_simls = cosine_similarity(tfidf,tfidf)
con_simls


array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.30411642, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.30411642, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

try one by one to get a similarity score

In [ ]:
#take by index 0
con_simls[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.15234058, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.11526394,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.06554154, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.07104787,
       0.        , 0.        , 0.        , 0.        , 0.     

In [ ]:
#sorting with ascending
score_pd = pd.Series(con_simls[0]).sort_values(ascending = False)
score_pd

0      1.000000
171    0.222890
5      0.152341
201    0.135102
175    0.125867
         ...   
143    0.000000
142    0.000000
141    0.000000
140    0.000000
429    0.000000
Length: 430, dtype: float64

In [ ]:
top_=list(score_pd.iloc[1:7].index)
top_

[171, 5, 201, 175, 14, 124]

In [ ]:
data = []
for i in top_:
    data.append(df_bar.iloc[i][0]+" - "+df_bar.iloc[i][2])


In [ ]:
data

['Afif - baik jujur berbudi pekerti',
 'Aidil - bermakna jujur',
 'Abisatya - seorang jujur',
 'Amin - dipercaya jujur',
 'Bagus - bermakna baik',
 'Husain - baik indah']

# Simulation System

In [ ]:
def recommendation(desc):

  user_response = pd.DataFrame({ 'nama':['-'],
                              'arti':['-'],
                               'arti_clean':desc})



  data = []
  df_fc = pd.concat([user_response, df_nodc]).reset_index()
  df_fcp = df_fc.drop("index", axis=1)
  TfidfVec= TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,)
  tfidf= TfidfVec.fit_transform(df_fcp['arti_clean'])
  con_simls = cosine_similarity(tfidf,tfidf)

  score = pd.Series(con_simls[0]).sort_values(ascending = False)
  top_ = list(score.iloc[1:7].index)
  for i in top_:
     data.append(df_fcp.iloc[i][0]+" - "+df_fcp.iloc[i][2])
  return data


In [ ]:
recommendation("selalu gembira senang menolong")

['Arjanta - selalu gembira',
 'Bagya - senang bahagia',
 'Fidaan - bermakna selalu membantu',
 'Astaguna - selalu berbuat kebaikan',
 'Andaru - wahyu kebahagiaan suka menolong dermawan memberi jala',
 'Abiyasa - seorang selalu mendekati sifat sifat terpuji']